# Analysis of the Experiments of Data Drift Detection methods being applied to Synthetic Data streams

#### Import data and set some helper functions

In [16]:
import pandas as pd

# pandas needs to read the consolidated_results.csv
df = pd.read_csv("comparison_results/consolidated_results.csv")

# we need to filter just the synthetic datasets
df = df[df["type_of_dataset"].str.contains("synthetic")]

In [17]:
def print_best_per_group(df, metric_to_optimize):
    """
    Prints the best results per group for each algorithm based on a specified metric.
    
    Args:
    - df (pd.DataFrame): The input dataframe containing data for all algorithms.
    - metric_to_optimize (str): The column name of the metric to optimize.
    """
    # Iterate through each unique algorithm in the dataframe
    for algorithm in df['algorithm'].unique():
        print(f"Algorithm: {algorithm}\n")
        
        # Slice the dataframe for the current algorithm
        df_algorithm = df[df['algorithm'] == algorithm]

        # Iterate through each unique scenario within the current algorithm
        for scenario in df_algorithm['scenario'].unique():
            print(f"  Scenario: {scenario}\n")

            df_scenario = df_algorithm[df_algorithm['algorithm'] == algorithm]
        
            # Group by specified columns and select the best row per group based on the metric_to_optimize
            best_per_group = (
                df_scenario.groupby(["drift_alignment_with_batch", "batch_size"], as_index=False)
                .apply(lambda group: group.loc[group[metric_to_optimize].idxmax()])
                .reset_index(drop=True)
            )
            
            # Print the desired columns for the best results
            print(best_per_group[["batch_size", "drift_alignment_with_batch", "technique", metric_to_optimize]])
            print("\n" + "="*80 + "\n")


In [18]:
import math
math.ceil(20000 / (3 * 2))

3334

## Quick look at the datasets

This notebook initially is looking at the experiment with dataset size = 80000, drift length = 20000, drifts in 3 features, with 2 repeating cycle of drifts. Each window: has drift_length / (n_features_with_drift * drift_repeats) 

The added drifts are all abrupt.

Considering batch size as 1000, this is how the sliding drift inside batches work:

### 5% of drift inside the batch
![5% of drift inside a batch](comparison_results/synthetic_dataset_with_parallel_drifts/feature_plots/synthetic_dataset_with_parallel_drifts_feature1_parallel_drifts_and_batches_1000_0.05.png)

### 50% of drift inside the first batch
![50% of drift inside a batch](comparison_results/synthetic_dataset_with_parallel_drifts/feature_plots/synthetic_dataset_with_parallel_drifts_feature1_parallel_drifts_and_batches_1000_0.5.png)

### 100% of drift inside the first batch
![100% of drift inside a batch](comparison_results/synthetic_dataset_with_parallel_drifts/feature_plots/synthetic_dataset_with_parallel_drifts_feature1_parallel_drifts_and_batches_1000_1.0.png)

### 100% of drift inside the first batch - parallel

![100% of drift inside a batch](comparison_results/synthetic_dataset_with_parallel_drifts/feature_plots/synthetic_dataset_with_parallel_drifts_all_features_parallel_drifts_and_batches_1000_1.0.png)

### 100% of drift inside the first batch - switching

![100% of drift inside a batch](comparison_results/synthetic_dataset_with_switching_drifts/feature_plots/synthetic_dataset_with_switching_drifts_all_features_switching_drifts_and_batches_1000_1.0.png)

In [19]:
df.dtypes

dataset                        object
batch_size                      int64
technique                      object
accuracy                      float64
precision                     float64
recall                        float64
f1                            float64
num_drifts                      int64
num_batches                     int64
auc                           float64
drift_alignment_with_batch    float64
scenario                       object
type_of_dataset                object
algorithm                      object
dtype: object

In [20]:
df.head()

,dataset,batch_size,technique,accuracy,precision,recall,f1,num_drifts,num_batches,auc,drift_alignment_with_batch,scenario,type_of_dataset,algorithm
0,synthetic_dataset_with_parallel_drifts_abrupt,1000,Base,0.642288,0.657160,0.642288,0.633694,0,80,0.642383,1.0,parallel_abrupt,synthetic,NB
1,synthetic_dataset_with_parallel_drifts_abrupt,1000,KS95,0.652825,0.658354,0.652825,0.649810,8,80,0.652883,1.0,parallel_abrupt,synthetic,NB
2,synthetic_dataset_with_parallel_drifts_abrupt,1000,KS90,0.655238,0.659365,0.655238,0.653027,10,80,0.655287,1.0,parallel_abrupt,synthetic,NB
3,synthetic_dataset_with_parallel_drifts_abrupt,1000,HD,0.646175,0.665440,0.646175,0.635645,4,80,0.646281,1.0,parallel_abrupt,synthetic,NB
4,synthetic_dataset_with_parallel_drifts_abrupt,1000,JS,0.646175,0.665440,0.646175,0.635645,4,80,0.646281,1.0,parallel_abrupt,synthetic,NB


# For each scenario and batch size, what technique performs better

#### Considering F1-Score

In [21]:
print_best_per_group(df ,"f1")

Algorithm: NB

  Scenario: parallel_abrupt

   batch_size  drift_alignment_with_batch technique        f1
0        1000                         1.0      KS95  0.666145
1        1500                         1.0      KS90  0.664691
2        2000                         1.0      KS95  0.663312


  Scenario: switching_incremental

   batch_size  drift_alignment_with_batch technique        f1
0        1000                         1.0      KS95  0.666145
1        1500                         1.0      KS90  0.664691
2        2000                         1.0      KS95  0.663312


  Scenario: switching_abrupt

   batch_size  drift_alignment_with_batch technique        f1
0        1000                         1.0      KS95  0.666145
1        1500                         1.0      KS90  0.664691
2        2000                         1.0      KS95  0.663312


  Scenario: no_drifts

   batch_size  drift_alignment_with_batch technique        f1
0        1000                         1.0      KS95  0.6

/var/folders/5f/sx30p29n23d2bq0_yfm62z4h0000gn/T/ipykernel_42356/3976136182.py:25: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda group: group.loc[group[metric_to_optimize].idxmax()])
/var/folders/5f/sx30p29n23d2bq0_yfm62z4h0000gn/T/ipykernel_42356/3976136182.py:25: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda group: group.loc[group[metric_to_optimize].idxmax()])
/var/folders/5f/

#### Considering AUC

In [22]:
print_best_per_group(df, "auc")

Algorithm: NB

  Scenario: parallel_abrupt

   batch_size  drift_alignment_with_batch technique       auc
0        1000                         1.0      KS95  0.668780
1        1500                         1.0      KS90  0.667027
2        2000                         1.0      KS95  0.665740


  Scenario: switching_incremental

   batch_size  drift_alignment_with_batch technique       auc
0        1000                         1.0      KS95  0.668780
1        1500                         1.0      KS90  0.667027
2        2000                         1.0      KS95  0.665740


  Scenario: switching_abrupt

   batch_size  drift_alignment_with_batch technique       auc
0        1000                         1.0      KS95  0.668780
1        1500                         1.0      KS90  0.667027
2        2000                         1.0      KS95  0.665740


  Scenario: no_drifts

   batch_size  drift_alignment_with_batch technique       auc
0        1000                         1.0      KS95  0.6

/var/folders/5f/sx30p29n23d2bq0_yfm62z4h0000gn/T/ipykernel_42356/3976136182.py:25: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda group: group.loc[group[metric_to_optimize].idxmax()])
/var/folders/5f/sx30p29n23d2bq0_yfm62z4h0000gn/T/ipykernel_42356/3976136182.py:25: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda group: group.loc[group[metric_to_optimize].idxmax()])
/var/folders/5f/

In [23]:
import json
import csv
from tabulate import tabulate

# Load the JSON data from the file
file_path = "/Users/lucashelfstein/Documents/Masters/EmpiricalAnalysisOfDataDrift/comparison_results/consolidated_drift_results.json"
with open(file_path, "r") as file:
    data = json.load(file)

# Function to compute performance metrics
def compute_metrics(true_drifts, detected_drifts):
    TP = len(detected_drifts & true_drifts)  # True positives
    FP = len(detected_drifts - true_drifts)  # False positives
    FN = len(true_drifts - detected_drifts)  # False negatives

    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return {
        "TP": TP,
        "FP": FP,
        "FN": FN,
        "Precision": round(precision, 3),
        "Recall": round(recall, 3),
        "F1-score": round(f1_score, 3)
    }

# Process each scenario
results = {}

for scenario in data:
    dataset_name = scenario["dataset"]
    drift_within_batch = scenario["drift_within_batch"]
    batch_size = scenario["batch_size"]
    scenario_id = f"{dataset_name}_drift_{drift_within_batch}_batch_{batch_size}"

    # Aggregate all synthetic drifts from different features
    true_drifts = set.union(*[set(batches) for batches in scenario["synthetic_drifts"].values()])

    # Compute performance metrics for each detection method
    metrics = {method: compute_metrics(true_drifts, set(detected_batches))
               for method, detected_batches in scenario["detected_drifts"].items()}

    # Store results
    results[scenario_id] = metrics

# Save results to CSV file
csv_file = "/Users/lucashelfstein/Documents/Masters/EmpiricalAnalysisOfDataDrift/comparison_results/drift_detection_results.csv"
with open(csv_file, mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["Scenario", "Method", "TP", "FP", "FN", "Precision", "Recall", "F1-score"])
    for scenario, metrics in results.items():
        for method, scores in metrics.items():
            writer.writerow([scenario, method, scores["TP"], scores["FP"], scores["FN"], scores["Precision"], scores["Recall"], scores["F1-score"]])

# Print results in structured table format
for scenario, metrics in results.items():
    print(f"\nScenario: {scenario}")
    table_data = [[method, scores["TP"], scores["FP"], scores["FN"], scores["Precision"], scores["Recall"], scores["F1-score"]]
                  for method, scores in metrics.items()]
    headers = ["Method", "TP", "FP", "FN", "Precision", "Recall", "F1-score"]
    print(tabulate(table_data, headers=headers, tablefmt="grid"))

print(f"Results saved to {csv_file}")



Scenario: synthetic_dataset_with_switching_drifts_abrupt_drift_1.0_batch_2000
+--------------+------+------+------+-------------+----------+------------+
| Method       |   TP |   FP |   FN |   Precision |   Recall |   F1-score |
+==============+======+======+======+=============+==========+============+
| ks_drifts    |   12 |    7 |    0 |       0.632 |      1   |      0.774 |
+--------------+------+------+------+-------------+----------+------------+
| ks_90_drifts |   12 |    8 |    0 |       0.6   |      1   |      0.75  |
+--------------+------+------+------+-------------+----------+------------+
| hd_drifts    |    6 |    7 |    6 |       0.462 |      0.5 |      0.48  |
+--------------+------+------+------+-------------+----------+------------+
| js_drifts    |    6 |    7 |    6 |       0.462 |      0.5 |      0.48  |
+--------------+------+------+------+-------------+----------+------------+

Scenario: synthetic_dataset_with_parallel_drifts_abrupt_drift_1.0_batch_1000
+------

# New results

In [71]:
import json
import csv
from tabulate import tabulate

# Load the JSON data from the file
file_path = "/Users/lucashelfstein/Documents/Masters/EmpiricalAnalysisOfDataDrift/comparison_results/consolidated_drift_results.json"
with open(file_path, "r") as file:
    data = json.load(file)

# Function to compute performance metrics
def compute_metrics(true_drifts, detected_drifts):
    TP = len(detected_drifts & true_drifts)  # True positives
    FP = len(detected_drifts - true_drifts)  # False positives
    FN = len(true_drifts - detected_drifts)  # False negatives

    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return {
        "TP": TP,
        "FP": FP,
        "FN": FN,
        "Precision": round(precision, 3),
        "Recall": round(recall, 3),
        "F1-score": round(f1_score, 3)
    }

# Process each scenario
results = {}

# Map for method mapping
method_mapping = {
    "ks_drifts": "KS95", 
    "ks_90_drifts": "KS90",
    "hd_drifts": "HD",
    "js_drifts": "JS"
}

for scenario in data:
    dataset_name = scenario["dataset"]
    drift_within_batch = scenario["drift_within_batch"]
    batch_size = scenario["batch_size"]
    scenario_id = f"{dataset_name}_drift_{drift_within_batch}_batch_{batch_size}"

    # Aggregate all synthetic drifts from different features
    true_drifts = set.union(*[set(batches) for batches in scenario["synthetic_drifts"].values()])

    # Compute performance metrics for each detection method, applying method_mapping
    metrics = {method_mapping.get(method, method): compute_metrics(true_drifts, set(detected_batches))
               for method, detected_batches in scenario["detected_drifts"].items()}

    # Store results
    results[scenario_id] = metrics


# Save results to CSV file
csv_file = "/Users/lucashelfstein/Documents/Masters/EmpiricalAnalysisOfDataDrift/comparison_results/drift_detection_results.csv"
with open(csv_file, mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["Scenario", "Method", "TP", "FP", "FN", "Precision", "Recall", "F1-score"])
    for scenario, metrics in results.items():
        for method, scores in metrics.items():
            writer.writerow([scenario, method, scores["TP"], scores["FP"], scores["FN"], scores["Precision"], scores["Recall"], scores["F1-score"]])

# Print results in structured table format
for scenario, metrics in results.items():
    print(f"\nScenario: {scenario}")
    table_data = [[method, scores["TP"], scores["FP"], scores["FN"], scores["Precision"], scores["Recall"], scores["F1-score"]]
                  for method, scores in metrics.items()]
    headers = ["Method", "TP", "FP", "FN", "Precision", "Recall", "F1-score"]
    print(tabulate(table_data, headers=headers, tablefmt="grid"))

print(f"Results saved to {csv_file}")


# Example: synthetic_dataset_with_switching_drifts_abrupt_drift_1.0_batch_2000
# string_format = f"{dataset_id}_{amount_of_drift_in_batch}_batch_{batch_size}"


Scenario: synthetic_dataset_with_parallel_drifts_incremental_drift_1.0_batch_2500
+----------+------+------+------+-------------+----------+------------+
| Method   |   TP |   FP |   FN |   Precision |   Recall |   F1-score |
+==========+======+======+======+=============+==========+============+
| KS95     |    4 |    3 |    0 |       0.571 |      1   |      0.727 |
+----------+------+------+------+-------------+----------+------------+
| KS90     |    4 |    3 |    0 |       0.571 |      1   |      0.727 |
+----------+------+------+------+-------------+----------+------------+
| HD       |    2 |    0 |    2 |       1     |      0.5 |      0.667 |
+----------+------+------+------+-------------+----------+------------+
| JS       |    2 |    0 |    2 |       1     |      0.5 |      0.667 |
+----------+------+------+------+-------------+----------+------------+

Scenario: synthetic_dataset_with_switching_drifts_incremental_drift_1.0_batch_2500
+----------+------+------+------+--------

In [74]:
import json
import csv
from tabulate import tabulate

# Load the JSON data from the file
file_path = "/Users/lucashelfstein/Documents/Masters/EmpiricalAnalysisOfDataDrift/comparison_results/consolidated_drift_results.json"
with open(file_path, "r") as file:
    data = json.load(file)

# Function to compute performance metrics
def compute_metrics(true_drifts, detected_drifts):
    TP = len(detected_drifts & true_drifts)  # True positives
    FP = len(detected_drifts - true_drifts)  # False positives
    FN = len(true_drifts - detected_drifts)  # False negatives

    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return {
        "TP": TP,
        "FP": FP,
        "FN": FN,
        "Precision": round(precision, 3),
        "Recall": round(recall, 3),
        "F1-score": round(f1_score, 3)
    }

# Process each scenario
results = {}

# Map for method mapping
method_mapping = {
    "ks_drifts": "KS95", 
    "ks_90_drifts": "KS90",
    "hd_drifts": "HD",
    "js_drifts": "JS"
}

for scenario in data:
    dataset_name = scenario["dataset"]
    drift_within_batch = scenario["drift_within_batch"]
    batch_size = scenario["batch_size"]
    scenario_id = f"{dataset_name}_drift_{drift_within_batch}_batch_{batch_size}"

    # Aggregate all synthetic drifts from different features
    true_drifts = set.union(*[set(batches) for batches in scenario["synthetic_drifts"].values()])

    # Compute performance metrics for each detection method, applying method_mapping
    metrics = {method_mapping.get(method, method): compute_metrics(true_drifts, set(detected_batches))
               for method, detected_batches in scenario["detected_drifts"].items()}

    # Store results
    results[scenario_id] = metrics

# Dataset mapping
dataset_mapping = {
    "synthetic_dataset_no_drifts": "SYN",
    "synthetic_dataset_with_parallel_drifts_abrupt": "SYN-PA",
    "synthetic_dataset_with_parallel_drifts_incremental": "SYN-PI",
    "synthetic_dataset_with_switching_drifts_abrupt": "SYN-SA",
    "synthetic_dataset_with_switching_drifts_incremental": "SYN-SI"
}

# Save results to CSV file with new format
csv_file = "/Users/lucashelfstein/Documents/Masters/EmpiricalAnalysisOfDataDrift/comparison_results/drift_detection_results.csv"
with open(csv_file, mode="w", newline="") as file:
    writer = csv.writer(file)
    
    # Updated headers
    writer.writerow(["Dataset", "Amount of Drift", "Batch Size", "Method", "TP", "FP", "FN", "Precision", "Recall", "F1-score"])
    
    for scenario, metrics in results.items():
        # Extract dataset, amount_of_drift_in_batch, and batch_size
        parts = scenario.split("_drift_")
        dataset_id = parts[0]  # e.g., synthetic_dataset_with_switching_drifts_abrupt
        rest = parts[1].split("_batch_")
        amount_of_drift_in_batch = rest[0]  # e.g., 1.0
        batch_size = rest[1]  # e.g., 2000

        # Map dataset ID
        dataset_name = dataset_mapping.get(dataset_id, dataset_id)  # Default to original if not found
        
        for method, scores in metrics.items():
            writer.writerow([dataset_name, amount_of_drift_in_batch, batch_size, method,
                             scores["TP"], scores["FP"], scores["FN"],
                             scores["Precision"], scores["Recall"], scores["F1-score"]])

# Print results in structured table format
for scenario, metrics in results.items():
    # Extract dataset, amount_of_drift_in_batch, and batch_size
    parts = scenario.split("_drift_")
    dataset_id = parts[0]
    rest = parts[1].split("_batch_")
    amount_of_drift_in_batch = rest[0]
    batch_size = rest[1]

    # Map dataset ID
    dataset_name = dataset_mapping.get(dataset_id, dataset_id)

    print(f"\nScenario: {dataset_name}, Amount of Drift: {amount_of_drift_in_batch}, Batch Size: {batch_size}")

    table_data = [
        [method, scores["TP"], scores["FP"], scores["FN"], scores["Precision"], scores["Recall"], scores["F1-score"]]
        for method, scores in metrics.items()
    ]
    
    headers = ["Method", "TP", "FP", "FN", "Precision", "Recall", "F1-score"]
    print(tabulate(table_data, headers=headers, tablefmt="grid"))


print(f"Results saved to {csv_file}")



Scenario: SYN-SI, Amount of Drift: 1.0, Batch Size: 2500
+----------+------+------+------+-------------+----------+------------+
| Method   |   TP |   FP |   FN |   Precision |   Recall |   F1-score |
+==========+======+======+======+=============+==========+============+
| KS95     |   13 |    5 |    1 |       0.722 |    0.929 |      0.813 |
+----------+------+------+------+-------------+----------+------------+
| KS90     |   13 |    5 |    1 |       0.722 |    0.929 |      0.813 |
+----------+------+------+------+-------------+----------+------------+
| HD       |    6 |    4 |    8 |       0.6   |    0.429 |      0.5   |
+----------+------+------+------+-------------+----------+------------+
| JS       |    6 |    4 |    8 |       0.6   |    0.429 |      0.5   |
+----------+------+------+------+-------------+----------+------------+

Scenario: SYN-PA, Amount of Drift: 1.0, Batch Size: 1000
+----------+------+------+------+-------------+----------+------------+
| Method   |   TP | 

In [78]:
import csv

def generate_latex_table(csv_file, batch_size):
    # Read CSV file
    data = []
    with open(csv_file, mode="r") as file:
        reader = csv.DictReader(file)
        for row in reader:
            if row["Batch Size"] == str(batch_size):  # Filter by batch size
                data.append(row)

    if not data:
        return f"No data found for batch size {batch_size}."

    # LaTeX Table Header
    latex_code = f"""\\begin{{table}}[ht]
\\caption{{Precision, Recall, and F1-score for a batch size of {batch_size}.}}
\\centering
\\begin{{tabular*}}{{\\textwidth}}{{@{{}}l c c c c@{{}}}}
\\hline \\hline
Dataset & Technique & Precision & Recall & F1-score \\\\ 
\\hline
"""

    # Fill in the table rows
    for row in data:
        dataset = row["Dataset"]
        method = row["Method"]
        precision = row["Precision"]
        recall = row["Recall"]
        f1_score = row["F1-score"]
        
        latex_code += f"{dataset} & {method} & {precision} & {recall} & {f1_score} \\\\\n"

    # Table Footer
    latex_code += """\\hline \\hline
\\end{tabular}
\\label{tab:metrics_""" + str(batch_size) + """}
\\end{table}"""

    return latex_code

# Example usage:
csv_file_path = "/Users/lucashelfstein/Documents/Masters/EmpiricalAnalysisOfDataDrift/comparison_results/drift_detection_results.csv"
batch_size_input = 2500
latex_table = generate_latex_table(csv_file_path, batch_size_input)

# Print or save the LaTeX table
print(latex_table)


\begin{table}[ht]
\caption{Precision, Recall, and F1-score for a batch size of 2500.}
\centering
\begin{tabular*}{\textwidth}{@{}l c c c c@{}}
\hline \hline
Dataset & Technique & Precision & Recall & F1-score \\ 
\hline
SYN-SI & KS95 & 0.722 & 0.929 & 0.813 \\
SYN-SI & KS90 & 0.722 & 0.929 & 0.813 \\
SYN-SI & HD & 0.6 & 0.429 & 0.5 \\
SYN-SI & JS & 0.6 & 0.429 & 0.5 \\
SYN-PI & KS95 & 0.571 & 1.0 & 0.727 \\
SYN-PI & KS90 & 0.571 & 1.0 & 0.727 \\
SYN-PI & HD & 1.0 & 0.5 & 0.667 \\
SYN-PI & JS & 1.0 & 0.5 & 0.667 \\
SYN-SA & KS95 & 0.706 & 0.857 & 0.774 \\
SYN-SA & KS90 & 0.706 & 0.857 & 0.774 \\
SYN-SA & HD & 1.0 & 0.286 & 0.444 \\
SYN-SA & JS & 1.0 & 0.286 & 0.444 \\
SYN-PA & KS95 & 0.571 & 1.0 & 0.727 \\
SYN-PA & KS90 & 0.571 & 1.0 & 0.727 \\
SYN-PA & HD & 1.0 & 1.0 & 1.0 \\
SYN-PA & JS & 1.0 & 1.0 & 1.0 \\
\hline \hline
\end{tabular}
\label{tab:metrics_2500}
\end{table}


In [1]:
import pandas as pd

results_file = "/Users/lucashelfstein/Documents/Masters/EmpiricalAnalysisOfDataDrift/comparison_results/consolidated_results.csv"

df_results = pd.read_csv(results_file)

In [5]:
df = df_results[['dataset', 'batch_size', 'technique','f1', 'auc']]

In [6]:
df[df.dataset == 'synthetic_dataset_with_switching_drifts_abrupt']

,dataset,batch_size,technique,f1,auc
340,synthetic_dataset_with_switching_drifts_abrupt,1000,Base,0.645339,0.649492
341,synthetic_dataset_with_switching_drifts_abrupt,1000,KS95,0.662664,0.664268
342,synthetic_dataset_with_switching_drifts_abrupt,1000,KS90,0.662664,0.664268
343,synthetic_dataset_with_switching_drifts_abrupt,1000,HD,0.657067,0.660815
344,synthetic_dataset_with_switching_drifts_abrupt,1000,JS,0.657067,0.660815
345,synthetic_dataset_with_switching_drifts_abrupt,1500,Base,0.643748,0.647577
346,synthetic_dataset_with_switching_drifts_abrupt,1500,KS95,0.656290,0.658158
347,synthetic_dataset_with_switching_drifts_abrupt,1500,KS90,0.657017,0.658693
348,synthetic_dataset_with_switching_drifts_abrupt,1500,HD,0.654753,0.657063
349,synthetic_dataset_with_switching_drifts_abrupt,1500,JS,0.654753,0.657063
